In [1]:
import gensim
import numpy as np
import pandas as pd
import multiprocessing
from gensim.models import KeyedVectors
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn import utils
from sklearn.utils import shuffle
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument

In [2]:
df1 = pd.read_csv('lib/dataset',index_col=0)
df2 = pd.read_csv('lib/dataset ahok',index_col=0)
df3 = pd.read_csv('lib/datasetfm',index_col=0)

In [3]:
dfAll=pd.concat([df1,df2,df3],sort=True)
dfAll = dfAll.replace(to_replace='None', value=np.nan).dropna()
dfAll = shuffle(dfAll)
dfAll.dropna(inplace=True)
dfAll.reset_index(drop=True,inplace=True)
dfAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18182 entries, 0 to 18181
Data columns (total 2 columns):
target    18182 non-null int64
text      18182 non-null object
dtypes: int64(1), object(1)
memory usage: 284.2+ KB


In [4]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [5]:
all_x = dfAll.text
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [6]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 18182/18182 [00:00<00:00, 797257.15it/s]


In [7]:
modelCbow = gensim.models.Word2Vec.load('lib/cbow_indo.word2vec')
modelSg = gensim.models.Word2Vec.load('lib/sg_indo.word2vec')

cbow

In [8]:
modelCbow.build_vocab([x.words for x in tqdm(all_x_w2v)],update=True)

100%|██████████| 18182/18182 [00:00<00:00, 488898.52it/s]


In [9]:
for epoch in tqdm(range(7200)):
    modelCbow.train(utils.shuffle([x.words for x in (all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    modelCbow.alpha -= 0.002
    modelCbow.min_alpha = modelCbow.alpha

100%|██████████| 7200/7200 [42:54<00:00,  2.42it/s]


In [10]:
modelCbow.wv.most_similar('bagus')

[('gosok', 0.9809937477111816),
 ('sejuk', 0.9759219884872437),
 ('hny', 0.9753035306930542),
 ('tulis', 0.970203161239624),
 ('confuse', 0.9695990681648254),
 ('tawar', 0.9695308804512024),
 ('rasa', 0.968677282333374),
 ('pansos', 0.966873824596405),
 ('iyaaa', 0.9654723405838013),
 ('s', 0.9654042720794678)]

sg

In [11]:
modelSg.build_vocab([x.words for x in tqdm(all_x_w2v)],update=True)

100%|██████████| 18182/18182 [00:00<00:00, 1008901.35it/s]


In [12]:
for epoch in tqdm(range(7200)):
    modelSg.train(utils.shuffle([x.words for x in (all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    modelSg.alpha -= 0.002
    modelSg.min_alpha = modelSg.alpha

100%|██████████| 7200/7200 [48:09<00:00,  3.26it/s] 


In [13]:
modelSg.wv.most_similar('bagus')

[('dapetnya', 0.9536737203598022),
 ('warning', 0.9430612921714783),
 ('cil', 0.9350965619087219),
 ('titip', 0.9349965453147888),
 ('solidaritas', 0.9345264434814453),
 ('ngejarah', 0.9328885078430176),
 ('irama', 0.931515634059906),
 ('pakcik', 0.9303817749023438),
 ('halang', 0.9300366044044495),
 ('kenyang', 0.9300069808959961)]